In [1]:
import os
from art.params import *

base_dir = LOCAL_DATA_PATH
train_dir = os.path.join(base_dir, "train")
validation_dir = os.path.join(base_dir, "validation")
test_dir = os.path.join(base_dir, "test")

In [6]:
import os
from PIL import Image

def is_image_valid(image_path):
    try:
        img = Image.open(image_path)
        img.verify()  # Verify that it is an image
        img.close()   # Close the image to avoid resource leaks

        # Reopen the image to ensure it can be read
        img = Image.open(image_path)
        img.load()    # Ensure the image can be fully loaded
        img.close()
        return True
    except (IOError, SyntaxError, OSError) as e:
        return False

def check_and_remove_invalid_images(base_dir):
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            file_path = os.path.join(root, file)
            if not is_image_valid(file_path):
                print(f"Removing invalid image: {file_path}")
                os.remove(file_path)

# Set the path to the directory containing your image dataset
dataset_dir = LOCAL_DATA_PATH

# Check and remove empty or invalid images
check_and_remove_invalid_images(dataset_dir)


Removing invalid image: /home/poloniki/.lewagon/art_data/train/Impressionism/Cafeneaua Lui Mamut.jpg
Removing invalid image: /home/poloniki/.lewagon/art_data/train/Art Informel/Von Goldfarben Und Blau 1953.jpg
Removing invalid image: /home/poloniki/.lewagon/art_data/train/Symbolism/Vilna Mostovaya Street.jpg
Removing invalid image: /home/poloniki/.lewagon/art_data/train/Realism/Autumn Landscape 1866.jpg


In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory



# Specify the batch size and image dimensions
batch_size = 32
img_height = 224
img_width = 224

# Load the training dataset with augmentation
train_dataset = image_dataset_from_directory(
    train_dir,

    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical',  # For multi-class classification
    labels='inferred'
)

# Load the validation dataset
validation_dataset = image_dataset_from_directory(
    validation_dir,

    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical',  # For multi-class classification
    labels='inferred'
)

# Create data augmentation layers
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomWidth(0.2),
    tf.keras.layers.RandomHeight(0.2)
])

# Prefetch the datasets for better performance
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)


Found 77293 files belonging to 28 classes.
Found 13828 files belonging to 28 classes.


In [11]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Create a sequential model and add the VGG16 model
model = Sequential([
    data_augmentation,          # Adding data augmentation directly into the model
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(28, activation='softmax')  # Number of classes inferred from dataset
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [13]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True

# Train the model
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=300,
    callbacks=[early_stopping],
)


Epoch 1/300


  83/2416 [>.............................] - ETA: 14:22 - loss: 6.7537 - accuracy: 0.0602

Corrupt JPEG data: 4015 extraneous bytes before marker 0xe2


 439/2416 [====>.........................] - ETA: 9:50 - loss: 4.6755 - accuracy: 0.0829

2024-06-05 23:18:47.418139: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


...

2416/2416 [==============================] - 128s 53ms/step - loss: 2.0553 - accuracy: 0.3627 - val_loss: 1.8759 - val_accuracy: 0.4125
Epoch 17/300
  86/2416 [>.............................] - ETA: 1:45 - loss: 2.0537 - accuracy: 0.3652

Corrupt JPEG data: 4015 extraneous bytes before marker 0xe2


 442/2416 [====>.........................] - ETA: 1:32 - loss: 2.0332 - accuracy: 0.3681

2024-06-06 00:11:04.692540: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


1312/2416 [===============>..............] - ETA: 51s - loss: 2.0268 - accuracy: 0.3685